# Analytic continuation of a scalar Green's function with known spectrum

In this example, we will analytically continue a Matsubara Green's function for which we known the exact corresponding spectral function using OmegaMaxEnt_TRIQS. To do so, we will start from a spectrum $A(\omega)$ that is a sum of gaussian functions, generate the corresponding Matsubara frequency Green's function using the spectral representation:

$$G(i\omega_n)=\int d\omega \frac{A(\omega)}{i\omega_n-\omega}\,,\qquad \text{(1)}$$

add noise to that function, and use OmegaMaxEnt_TRIQS to get back the real frequency Green's function. First, we import the modules we need:

In [ ]:
import OmegaMaxEnt_TRIQS as OT
from math import ceil, exp, sqrt, pi
from pytriqs.dos import HilbertTransform, DOSFromFunction  #to compute G from A
from pytriqs.gf import *
import numpy as np
from pytriqs.plot.mpl_interface import oplot,plt  #to plot the result
import os
import shutil as su

First, because the program produces files, it is better to create a working directory to group all the files:

In [ ]:
dir_name="G_iomega_1_ouput"
if not os.path.exists(dir_name):
    os.mkdir(dir_name)
os.chdir(dir_name)

Then, we define peaks positions, standard deviations, and weights and an appropriate frequency range outside of which the spectrum essentially vanishes:

In [ ]:
mw=[-2, 0, 1]
sd=[1, 0.1, 0.4] 
wgt=[1, 1, 1]
Wgts= sum(wgt)
Npks=len(mw)
wmin=-10
wmax=10

and define a function returning a value of the spectrum:

In [ ]:
def spectr_val(w):
    v = 0
    for i in range(0,Npks):
        v = v + (wgt[i] / sd[i]) * exp(-(w - mw[i]) * (w - mw[i]) / (2 * sd[i] * sd[i]))

    return v / (Wgts * sqrt(2 * pi))

which we have normalized. Now, we define an appropriate frequency step much smaller than the smallest peak width, and create a [DOSFromFunction][DFF] object:

[DFF]: https://triqs.github.io/triqs/master/reference/lattice_tools/dos.html?highlight=dosfromfunction

In [ ]:
Dw=min(sd)/10
Npts=int((wmax-wmin)/Dw+1)
d = DOSFromFunction(spectr_val, wmin, wmax, Npts)

Then, we define the temperature and a maximum Matsubara frequency much larger the spectrum's cutoff frequency $W_{max,A}$, i.e. the maximum $|\omega|$ such that $A(\omega)>\epsilon $, where $\epsilon$ is very small:

In [ ]:
beta=50
wnmax=30
n_freq=int(ceil(beta*wnmax/(2*pi)))

The fact that $wnmax\gg W_{max,A}$ is very useful to $\Omega MaxEnt$ because it allows it to extract moments, which provide the mean frequency and the standard deviation of the spectrum, that are then used to define well suited default model and frequency grid. 

We can now compute the Green's function with the function [HilbertTransform][HT]:

[HT]: https://triqs.github.io/triqs/master/reference/lattice_tools/hilbert.html

In [ ]:
G = GfImFreq(indices=[0], beta=beta, n_points=n_freq)
Sigma0 = GfImFreq(indices=[0], beta=beta, n_points=n_freq) 
Sigma0.zero()
G << HilbertTransform(d)(Sigma=Sigma0)
G=G[0,0]

Now, let us define error vectors for a constant relative error:

In [ ]:
err=1e-5
err_abs_min=1e-8
errGr=err * np.absolute(G.data.real)
# here we make sure the absolute error on the real part is not smaller than err_abs_min
for i in range(0,2*n_freq):
    if errGr[i]<err_abs_min:
        errGr[i] = err_abs_min
errGi=err * np.absolute(G.data.imag)
ERRG=errGr+1j*errGi

where we have set a minimal absolute error for the real part because it can vanish at some frequencies, or completely if the spectrum is symmetric. We have also defined the vector *ERRG* that has the same type and shape as *G.data* which will be passed to the function *computeGfReFreq()*.

Then, we add normally distributed noise to _G_ using *errGr* and *errGi* as standard deviation vectors:

In [ ]:
G.data.real =G.data.real + errGr * np.random.randn(2*n_freq)
G.data.imag =G.data.imag + errGi * np.random.randn(2*n_freq)

and run the function *compute_GfReFreq()* from the OmegaMaxEnt_TRIQS module:

In [ ]:
GR=OT.compute_GfReFreq(G, ERR=ERRG, interactive_mode=False, displ_alpha_min_figs=False)

Here we have set interactive_mode to *False* because the program's output figures cannot be displayed when it is called from a notebook. However, in the terminal where this notebook is running, you can see the text output of $\Omega MaxEnt$. We also have set *displ_alpha_min_figs=False* to produce only the figures related to the optimal result (labeled as the *optimal* $\alpha$). Otherwise, you would also see the result at the lowest $\alpha$, which is in the "noise fitting" range of $\alpha$.

Here, because the Green's function is scalar, script and data files were saved to display the figures after execution with the function *display_figures()* of the OmegaMaxEnt_TRIQS module:

In [ ]:
%matplotlib inline

In [ ]:
OT.display_figures()

The figure showing the result should be the two last three ones, titled **Retarded Green function in time**, **Retarded Green function in frequency**, and **Spectrum at optimal $\alpha$**. In the figures from the preprocessing, you see the curves of $M_0$ to $M_3$, showing the value of the moments:

$$M_n=\int d\omega\,\omega^n A(\omega)\qquad (2)$$

as a function of the starting frequency in the fit of

$$G_{asym}(i\omega_n)=\frac{M_0}{i\omega_n}+\frac{M_1}{(i\omega_n)^2}+\frac{M_2}{(i\omega_n)^3}+\ldots \qquad (3)$$

to $G(i\omega_n)$. The link between the high frequency expansion and the moments is easy to obtain by developping Eq. (1) at high $\omega_n$. From the plateaus in the curves, we clearly see that the expansion is valid for $\omega_n$ larger than a certain value $W_{max,A}$, which corresponds to the cutoff frequency of the spectrum mentioned above. From the moments we know that the spectrum is centered at $\langle\omega\rangle=M_1/M_0$ and has a standard deviation $\sigma=\sqrt{M_2/M_0-(M_1/M_0)^2}$, which are used to define the grid and the default model.

In the figures above, the last preprocessing figure is the default model. Then, in the result figures, we have the errors on the moments, normalized by the standard deviations (parameters *err_norm* and *default_error_M*). Then, we have quantities plotted as functions of $\log\alpha$: $\log\chi^2$, which shows the three different regimes of MaxEnt: *default model* (flat region at high $\alpha$), *information fitting* (steep region), *noise fitting* (flat region at low $\alpha$), the curvature ($\kappa$) of $\log\chi^2$ vs $\gamma\log\alpha$, which is used to determine the optimal $\alpha$ (the maximum of $\kappa$), the entropy *S*, and finally, the spectrum at sample frequencies, which is useful to determine how reliable different parts of the spectrum are, based on their stability with respect to $\log\alpha$.

Then we have the normalized deviations for the real and the imaginary part of $G$ and their autocorrelations. If the output Green's function, given by Eq. (1) using the optimal spectrum $A_{opt}(\omega)$, is a good fit to the input Green's function, the deviations curves must look like pure noise and their autocorrelations look like noisy Kronecker deltas. See the $\Omega MaxEnt$ [user guide][OME_user_guide] for more discussion on this subject.

[OME_user_guide]: https://www.physique.usherbrooke.ca/MaxEnt/index.php?title=User_Guide

Now we define the spectrum on a grid and compare the imaginary part of GR with the exact spectrum:

In [ ]:
%matplotlib notebook

In [ ]:
wl=-5
wr=5
dw=0.01
Nw=int((wr-wl)/dw)+1
w=dw*np.array(range(0,Nw))+wl
Aw=np.zeros(Nw)
for i in range(0,Nw):
    Aw[i]=spectr_val(w[i])

plt.plot(w,Aw,label='exact spectrum')
oplot(-GR.imag/pi, x_window = (-5,5))
plt.ylabel('')
plt.legend()

Note that this last figure is interactive because we have used the magic command **%matplotlib notebook** instead of **%matplotlib inline**. For example, you can use the zoom tool to see better the difference between the exact result and the maxent result.

The resulting spectrum should be close to the exact spectrum. You can see the difference however by zooming on some parts of the spectrum. This is because the error is small. 

The next input will destroy the working directory.

In [ ]:
os.chdir("..")
su.rmtree(dir_name)

## Exercise

Start from two different spectra $A_1(\omega)$ and $A_2(\omega)$ defined by
$$
mw1=[-2, 0, 1]\\
sd1=[1, 0.2, 0.5]\\
wgt1=[1,1,1]\\
$$
and
$$
mw2=[-1, 1.5]\\
sd2=[0.3, 0.7]\\
wgt2=[1,1]
$$

and define a $2\times 2$ diagonal matrix Green's function at $\beta=50$. Then, compute a non-diagonal Green's function by applying a rotation of $\pi/8$, add normally distributed noise of constant magnitude of $10^{-4}$ and perform the analytic continuation of that Green's function matrix. Finally, by using the definition of the spectral matrix:

$$\mathbf{A}(\omega)=\frac{i}{2\pi}\left[\mathbf{G}^R(\omega)-\left(\mathbf{G}^R(\omega)\right)^\dagger\right]\,,$$

compare the resulting three distinct spectra with the ones obtained from rotating the diagonal matrix with $A_1(\omega)$ and $A_2(\omega)$ on its diagonal.